<a href="https://colab.research.google.com/github/hunorxilard/onkutfel_2020-21/blob/szili_dolga/Onkutfel_2020_kodok.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#downloading and unzipping images
!wget https://github.com/hunorxilard/onkutfel_2020-21/blob/main/real_and_fake_face_small.zip?raw=True
!wget https://github.com/hunorxilard/onkutfel_2020-21/blob/main/half_test.zip?raw=True
!unzip ./half_test.zip?raw=True
!unzip ./real_and_fake_face_small.zip?raw=True

In [ ]:
#initializing for preprocessing
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import random

In [ ]:
#transfer images into useable training data
training_data = []

def create_training_data():
  for category in CATEGORIES:
    path=os.path.join(DATADIR,category)
    class_num = CATEGORIES.index(category)
    for img in os.listdir(path):
      img_array = cv2.imread(os.path.join(path,img),cv2.IMREAD_GRAYSCALE)
      training_data.append([img_array,class_num])

DATADIR = "/content/real_and_fake_face_small" #ez collabon mukodik, at kell irni, ha desktopon kell futtatni
CATEGORIES = ["training_fake","training_real"]
#DATADIR = "/content/half_test"
#CATEGORIES = ["fake","real"]

training_data = []
create_training_data()
random.shuffle(training_data)

X=[]
y=[]

for features, label in training_data:
  X.append(features)
  y.append(label)

X=np.array().reshape(-1,150,150,1) #a tf ilyen formatumban fogad el

#normalize data
X = X/255.0
y = np.asarray(y) #ez nem normalizalas, a tf csak ezt a formatumot fogadja el

In [ ]:
#initialize for training
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
import numpy as np
import datetime, os

#set up tensorboard
%load_ext tensorboard

In [ ]:
#start tensorboard
%tensorboard --logdir logs #ezen belul lefrissitve nyomon lehet kovetni a tanulast

In [ ]:
#optimizing method
dense_layers = [0,1]
layer_sizes = [32]
conv_layers = [2] #ezek a hiperparameterek, amik atirandok

for conv_layer in conv_layers:
  for layer_size in layer_sizes:
    for dense_layer in dense_layers:
      logdir = os.path.join("logs", "{}-conv-{}-nodes-{}-dense".format(conv_layer, layer_size, dense_layer))
      tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
      
      model = Sequential()
#konvolucios retegek pooling-al:
      model.add( Conv2D( layer_size, (3,3), input_shape = X_shape[1:] ) )
      model.add( Activation( "relu" ) )
      model.add( MaxPooling2D( pool_size = (2,2) ) )

      for l in range(conv_layer-1):
        model.add( Conv2D( layer_size, (3,3) ) )
        model.add( Activation( "relu" ) )
        model.add( MaxPooling2D( pool_size = (2,2) ) )
#lapositas a suru retegeknek:
      model.add( Flatten() )
#suru retegek:
      for l in range(dense_layer):
        model.add( Dense(256) )
        model.add( Activation("relu") )
        model.add( Dropout(0.2) )
#output reteg:
      model.add( Dense(1) )
      model.add( Activation( "sigmoid" ) )
#optimalizalasi parameterek:
      model.compile(loss = "binary_crossentropy",
                      optimizer = "adam",
                      metrics = ["accuracy"])
      model.fit(X, y, batch_size=32, epochs = 6, validation_split=0.15, callbacks=[tensorboard_callback])

In [ ]:
#save and load model weights:
model.save_weights('./final_model_weights')
#model = create_model
#model.load_weights("./final_model_weights")